In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\tonju\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# Mars news URL of page to be scraped
news_url = 'https://redplanetscience.com/'

browser.visit(news_url)

html = browser.html

news_soup = BeautifulSoup(html, 'html.parser')

## NASA Mars News

In [4]:
# Retrieve the latest news title and paragraph
# Apply inspect to identify class for news and title and save them to a variable
title = news_soup.find_all('div', class_='content_title')[0].text
paragraph = news_soup.find_all('div', class_='article_teaser_body')[0].text

print(title)
print("--------------------------------------------------------------------")
print(paragraph)

IndexError: list index out of range

## JPL Mars Space Images - Featured Image

In [ ]:
# Mars Image to be scraped
jpl_url = 'https://spaceimages-mars.com/'
image_url = 'https://spaceimages-mars.com/?search=&category=Mars'

browser.visit(image_url)

html = browser.html

images_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Retrieve featured image link
image_path = images_soup.find_all('img')[3]["src"]
featured_image_url = jpl_url + image_path
featured_image_url = featured_image_url.replace(" ", "%20")
print(featured_image_url)

## Mars Facts

In [ ]:
facts_url = 'https://galaxyfacts-mars.com'
tables = pd.read_html(facts_url)
tables   

In [ ]:
# tables = tables.drop(['Earth'], axis=1)
# tables.head()

In [ ]:
mars_facts_df = tables[0]
mars_facts_df.columns = ["Description", "Mars facts", 'Earth facts']
# mars_facts_df = mars_facts("Mars - Earth Comparison", "Mars")
mars_facts_df


In [ ]:
mars_html_table = mars_facts_df.to_html()
mars_html_table

In [ ]:
mars_html_table.replace('\n', '')

In [ ]:
print(mars_html_table)

## Mars Hemispheres

In [ ]:
url = 'https://marshemispheres.com/'
soup=BeautifulSoup(requests.get(url).text)
links=soup.find_all('a')
urls=[x['href'] for x in links]

for x in urls:
    print(urls)

In [ ]:
usgs_url = 'https://astrogeology.usgs.gov'
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hemispheres_url)

hemispheres_html = browser.html

hemispheres_soup = BeautifulSoup(hemispheres_html, 'html.parser')

In [ ]:
all_mars_hemispheres = hemispheres_soup.find('div', class_='collapsible results')
mars_hemispheres = all_mars_hemispheres.find_all('div', class_='item')

hemisphere_image_urls = []

# Iterate through each hemisphere data
for i in mars_hemispheres:
    # Collect Title
    hemisphere = i.find('div', class_="description")
    title = hemisphere.h3.text
    
    # Collect image link by browsing to hemisphere page
    hemisphere_link = hemisphere.a["href"]    
    browser.visit(usgs_url + hemisphere_link)
    
    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

    # Create Dictionary to store title and url info
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = image_url
    
    hemisphere_image_urls.append(image_dict)

print(hemisphere_image_urls)

In [ ]:
mars_dict = {
        "news_title": title,
        "paragraph": paragraph,
        "featured_image_url": featured_image_url,
        "fact_table": str(mars_html_table),
        "hemisphere_images": hemisphere_image_urls
    }

In [ ]:
mars_dict

In [ ]:
browser.quit()